### Pendulum Swing-up Reinforcement Learning

In [1]:
from IPython.display import HTML
from IPython.display import display
import numpy as np

In [2]:
# Taken from https://stackoverflow.com/questions/31517194/how-to-hide-one-specific-cell-input-or-output-in-ipython-notebook
tag = HTML('''<script>
code_show=false; 
function code_toggle() {
    if (code_show){
        $('div.cell.code_cell.rendered.selected div.input').hide();
    } else {
        $('div.cell.code_cell.rendered.selected div.input').show();
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
To show/hide this cell's raw code input, click <a href="javascript:code_toggle()">here</a>.''')
display(tag)

<img style="float: centre;" src="compoundPendulum.png" alt="Compound Pendulum" width="200" height="200"/>

Figure shows a compound pendulum of mass *m*. Let $\hat{i}$, $\hat{j}$ and $\hat{\lambda}$ be the unit vectors in the respective directions shown, $dx$ is an elemental length at a distance $x$ from the pivot point. A torque of $u$ is applied on the pivot and $L$ is the length of the pendulum. The mass per unit length is $m/L$ and the mass of $dx$ will be $\frac{m}{L}dx$

### Derivation of Equation of motion

$$ r = x \hat{\lambda}$$ where $\hat{\lambda} =  \sin\theta \hat{i} - \cos\theta \hat{j}$

$$
\begin{align}
    \dot{r} &= x \dot{\lambda} \\ 
    &= x(\dot{\theta} \hat{k} \times \lambda) \\
    &= x \dot{\theta} \hat{n}
\end{align}
$$

#### Kinectic Energy, 

$$
\begin{align}
    T &= \frac{1}{2}\int_0^l \dot{r}~.~\dot{r} dm \\
      &= \frac{1}{2}\int_0^l x^2 \dot{\theta}^2 dx \frac{m}{L} \\
      &= \frac{1}{6} m l^2 \dot{\theta}^2
\end{align}
$$

#### Potential Energy

$$U = -mg\frac{l}{2}\cos\theta$$

#### Lagrangian

$$
\begin{align}
    L &= T - U \\
      &= \frac{1}{6} ml^2 \dot{\theta}^2 + mg\frac{l}{2}\cos\theta
       \frac{\partial L}{\partial \theta} &= -mg\frac{l}{2}\sin\theta \\
    \frac{\partial L}{\partial \dot{\theta}} &= \frac{1}{3}ml^2\dot{\theta}^2 \\
    \frac{d}{dt}\Big(\frac{\partial L}{\partial \dot{\theta}}\Big) &= \frac{1}{3}ml^2\ddot{\theta} 
\end{align}
$$

#### The Equation of motion becomes

$$
\begin{align}
    \frac{d}{dt}\Big(\frac{\partial L}{\partial \dot{\theta}}\Big) - \frac{\partial L}{\partial \theta} &= u \\
    \frac{1}{3}ml^2\ddot{\theta} + mg\frac{l}{2}\sin\theta &= u \\
    \ddot{\theta} + \frac{3g}{2l} \sin\theta &= \frac{3u}{ml^2}
\end{align}
$$

#### if there is a damping term, then the equation of motion becomes

 $$ \ddot{\theta} + c\dot{\theta} +\frac{3g}{2l} \sin\theta = \frac{3u}{ml^2} $$

#### From finite difference approximation

$$
\begin{align}
    \frac{\dot{\theta}_{t+1} - \dot{\theta}_{t}}{dt} &= -\frac{3g}{2l} \sin\theta + \frac{3u}{ml^2}\\
    \dot{\theta}_{t+1} &= \dot{\theta}_{t} + dt \Big(-\frac{3g}{2l} \sin\theta + \frac{3u}{ml^2}\Big)\\
    \theta_{t+1} &= \theta_t + dt ~ \dot{\theta}_{t+1}
\end{align}
$$

[Openai Gym pendulum environment](https://github.com/openai/gym/blob/master/gym/envs/classic_control/pendulum.py)

In [3]:
def forwardDynamics(self, u):
    th, thdot = self.state  # th := theta

    g = self.g  # gravity = 9.81
    m = self.m  # mass
    l = self.l  # length
    dt = self.dt # time increment

    u = np.clip(u, -self.max_torque, self.max_torque)[0]
    
    newthdot = thdot + (-3 * g / (2 * l) * np.sin(th + np.pi) + 3. / (m * l ** 2) * u) * dt
    newth = th + newthdot * dt
    newthdot = np.clip(newthdot, -self.max_speed, self.max_speed)
     
    self.state = np.array([newth, newthdot])
    return newth, newthdot

In [4]:
# Normalization is carried out for neural networks to learn faster. These are observations.
def _get_obs(self):
    theta, thetadot = self.state
    return np.array([np.cos(theta), np.sin(theta), thetadot])

The maximum reward that you can obtain is 0. At either side of the vertical position, the reward goes negative. So the aim is to hold the pendulum vertical which means the angle $\theta$ and angular velocity $\dot{\theta}$ should both be 0 with minimal control effort

$$R = r_t(s_t, a_t)$$

In [5]:
def angle_normalize(x):  # to keep angles in the range -pi to pi
    return (((x+np.pi) % (2*np.pi)) - np.pi)

# When pendulum is vertical, theta = 0.
def reward(self, u):
    theta, thetadot = self.state
    reward = -(angle_normalize(th) ** 2 + .1 * thdot ** 2 + .001 * (u ** 2))
    return reward

#### Other reward functions

<img style="float: centre;" src="rewards.png" alt="" width="300" height="300"/>

In [6]:
display(tag)
# reward = -(s_t - s_T)**2 + u.u

In [7]:
# done and info are useful for other RL environments but is included 
# here to have a consistency. Usually done in this context means if the pendulum 
# vertical or not (always boolean). Info corresponds to any useful info corresponding
# to the environment that you want to return if there exists

def step(self, u):
    obs = self._get_obs()
    reward = self.reward(u)
    done = False  
    info = {}
    return obs, reward, done, info

In [8]:
# RL cannot find a solution to the problem in the first run itself. It requires a 
# lot of explorations for get something meaningful. So after exploring for sometime
# it has to be set back to some initial configurations. Reset() function does that

def reset(self):
    high = np.array([np.pi, 1])
    self.state = self.np_random.uniform(low=-high, high=high)
    self.last_u = None
    return self._get_obs()

In [9]:
def render(self):
    # Write functions to visualize()
    pass

##### Therefore the basic components of an RL environment are

In [10]:
class RobotEnv:
    def __init__(self, ):
        pass
    
    def _get_obs_(self, ):
        pass
    
    def step(self, u):
        pass
    
    def reset(self):
        pass
    
    def render(self):
        pass

For complex system, hand coding of forward dynamics and visualization are very difficult. Therefore simulators are used. Most popular simulators are Mujoco, pyBullet, ROS-Gazebo etc. These simulators need a robot description file, either a URDF or xml (for Mujoco)

[Openai gym environments](https://gym.openai.com/envs/#classic_control) are very popular for RL tasks. 

```pip install gym ```

In [11]:
import gym

In [12]:
env = gym.make('Pendulum-v0')  # gym.make('env_name-version')

Aim of RL: To maximize sum of expected future rewards by discounting

In [13]:
display(tag) # to hide/show this cell
idx = 500
cum_rewards = np.zeros(500)
env.reset()
for i in range(idx):
    action = env.action_space.sample()
    next_state, reward, done, info = env.step(action)
    cum_rewards[i] = reward
    env.render()

env.close()